In [6]:
import pandas as pd
import geopandas as gpd
import numpy as np

# --- 1. 데이터 불러오기 ---
try:
    df_biz = pd.read_csv("../data_csv/11_11/춘천시업종_좌표포함.csv")
    df_bus = pd.read_csv("../data_csv/11_10/춘천시_버스정류장노선정보_20250212.csv",encoding = 'CP949')
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 파일명을 확인해 주세요.")
    # 실제 환경에서는 여기서 처리를 중단해야 할 수 있습니다.
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
    # 실제 환경에서는 여기서 처리를 중단해야 할 수 있습니다.

# --- 2. 데이터 전처리 ---
# 업종 데이터: 위도/경도 값이 없는(NaN) 행 제거
df_biz_clean = df_biz.dropna(subset=['위도', '경도']).copy()
print(f"업종 데이터: {len(df_biz)} 행 -> 결측치 제거 후 {len(df_biz_clean)} 행")

# 버스정류장 데이터: 중복되는 정류장(동일 위도/경도/이름)을 하나로 통합
df_bus_unique = df_bus.drop_duplicates(subset=['정류장명', '위도', '경도']).copy()
print(f"버스정류장 데이터: {len(df_bus)} 행 -> 중복 제거 후 {len(df_bus_unique)} 행")

# 전처리 후 데이터가 비었는지 확인
if df_biz_clean.empty or df_bus_unique.empty:
    print("전처리 후 데이터가 비어있어, 조인을 진행할 수 없습니다.")
else:
    # --- 3. GeoDataFrame 생성 (위/경도 좌표계: EPSG:4326) ---
    # geopandas가 공간 연산을 할 수 있도록 GeoDataFrame으로 변환합니다.
    try:
        # GeoDataFrame 생성 (경도=x, 위도=y)
        gdf_biz = gpd.GeoDataFrame(
            df_biz_clean, 
            geometry=gpd.points_from_xy(df_biz_clean.경도, df_biz_clean.위도),
            crs="EPSG:4326"  # WGS84 (위도/경도) 좌표계
        )
        
        gdf_bus = gpd.GeoDataFrame(
            df_bus_unique, 
            geometry=gpd.points_from_xy(df_bus_unique.경도, df_bus_unique.위도),
            crs="EPSG:4326"
        )
    except Exception as e:
        print(f"GeoDataFrame 생성 중 오류 발생: {e}")
        print("경도 또는 위도 컬럼에 숫자(float)가 아닌 값이 포함되어 있을 수 있습니다.")

    # --- 4. 좌표계 변환 (미터(m) 기반 좌표계: EPSG:5186) ---
    # '유사함'의 기준이 될 거리를 미터(m) 단위로 정확하게 계산하기 위해
    # 위/경도(도 단위) 좌표계를 미터(m) 기반의 좌표계로 변환합니다.
    # EPSG:5186은 한국 중부원점(GRS80) 기준의 미터 좌표계입니다.
    gdf_biz_proj = gdf_biz.to_crs("EPSG:5186")
    gdf_bus_proj = gdf_bus.to_crs("EPSG:5186")

    # --- 5. '유사' 조인 (공간 조인) ---
    # '유사함'의 기준(거리)을 100m로 설정합니다.
    distance_threshold = 100  # 100 미터

    print(f"\n--- {distance_threshold}m 이내 'inner' 공간 조인 시작 ---")
    
    # sjoin_nearest: 가장 가까운 대상을 찾아 조인
    # how='inner': 100m 이내에 대상이 있는 경우에만 조인 (결과에 남김)
    # max_distance: '유사함'의 최대 허용 거리 (이 거리보다 멀면 조인 안 됨)
    merged_gdf = gpd.sjoin_nearest(
        gdf_biz_proj, 
        gdf_bus_proj, 
        how='inner', 
        max_distance=distance_threshold,
        lsuffix='업종',  # 왼쪽(업종) DataFrame의 컬럼 접미사
        rsuffix='정류장' # 오른쪽(버스정류장) DataFrame의 컬럼 접미사
    )

    # --- 6. 결과 확인 및 저장 ---
    if merged_gdf.empty:
        print(f"결과: {distance_threshold}m 이내에 위치한 업종과 버스정류장이 없습니다.")
    else:
        print(f"결과: 총 {len(merged_gdf)} 건의 '업종-정류장' 연결을 찾았습니다.")
        
        # geopandas의 geometry 컬럼은 CSV 저장 시 불필요하므로 일반 DataFrame으로 변환
        merged_df = pd.DataFrame(merged_gdf.drop(columns='geometry'))
        
        print("\n--- 병합된 데이터 (처음 5행) ---")
        print(merged_df.head())
        
        # 결과 파일 저장
        output_filename = "../data_csv/11_12/업종_100m이내_버스정류장_목록.csv"
        merged_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n'{output_filename}' 파일로 저장되었습니다.")

업종 데이터: 34 행 -> 결측치 제거 후 32 행
버스정류장 데이터: 15780 행 -> 중복 제거 후 1782 행

--- 100m 이내 'inner' 공간 조인 시작 ---
결과: 총 22 건의 '업종-정류장' 연결을 찾았습니다.

--- 병합된 데이터 (처음 5행) ---
   Unnamed: 0  업종      업소명                     주소           연락처      위도_업종  \
0           0  한식       청재     강원도 춘천시 서면 박사로 927  033-255-2867  37.898465   
1           1  한식    두리감자탕      강원도 춘천시 후평동 748-4  033-244-2847  37.888485   
2           2  한식  약사촌 닭갈비       강원도 춘천시 공지로384-9  033-256-9255  37.871951   
3           3  한식      산마루  강원도 춘천시 백령로 171-1(1층)  033-241-0769  37.876259   
4           4  한식       청재     강원도 춘천시 서면 박사로 927  033-255-2867  37.898465   

        경도_업종  index_정류장          노선번호           노선  정류장순서        정류장  \
0  127.693761       4942  서면1-1(서면104)  250050306.0   22.0  250002168   
1  127.751478       1436             9  250000900.0    4.0  250026602   
2  127.724553         67             1  250000100.0   68.0  250002028   
3  127.745220         12             1  250000100.0   13.0  250002238   
4  127.6

In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np

# --- 1. 데이터 불러오기 ---
try:
    df_biz = pd.read_csv("../data_csv/11_11/춘천시업종_좌표포함.csv")
    df_bus = pd.read_csv("../data_csv/11_10/춘천시_버스정류장노선정보_20250212.csv", encoding ='CP949')
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 파일명을 확인해 주세요.")
    # 실제 환경에서는 여기서 처리를 중단해야 할 수 있습니다.
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
    # 실제 환경에서는 여기서 처리를 중단해야 할 수 있습니다.
df_bus

,노선번호,노선,정류장순서,정류장,정류장명,경도,위도,데이터기준일
0,1,250000100.0,1.0,265000793,상공회의소,127.75267,37.89832,2025-02-12
1,1,250000100.0,2.0,250026779,장학해온채A,127.75391,37.89731,2025-02-12
2,1,250000100.0,3.0,250026778,장학교차로,127.75555,37.89383,2025-02-12
3,1,250000100.0,4.0,250026830,장학부영A,127.75278,37.89642,2025-02-12
4,1,250000100.0,5.0,250001211,후평동종점,127.74865,37.89326,2025-02-12
...,...,...,...,...,...,...,...,...
15775,906(종점송암리),250090601.0,120.0,250002125,세경3차,127.74370,37.88438,2025-02-12
15776,906(종점송암리),250090601.0,121.0,250026643,강원도시가스,127.74792,37.88586,2025-02-12
15777,906(종점송암리),250090601.0,122.0,250001874,정부합동청사,127.74964,37.88843,2025-02-12
15778,906(종점송암리),250090601.0,123.0,250001378,기계공고,127.74911,37.89186,2025-02-12
